In [1]:
import csv
import time
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import datetime
# import time

In [2]:
file = open("Feature_mapping.csv", "w", encoding="utf8")
f = csv.writer(file)

# Feature columns + label for each Userid
f.writerow(["Userid", "Running Morning", "Running Afternoon", "Running Evening", "Running Night","ADHD"])

69

In [3]:
df = pd.read_csv("Activity_clean.csv")
df = df.sort_values(by=['Userid','senseStartTimeMillis'],ascending=[True,True])

# get the list of ordered ids
ids = df["Userid"].unique()

In [4]:
# dictionary of ADHD labels for each user id
ADHD_dict = {}
baseline = pd.read_csv("sassema baseline 3.16.2023.csv")
baseline['ID'] = baseline['ID'].fillna(0)

for index in baseline.index:
    if index > 1:
        if baseline["ID"][index] != 0:
            if baseline["ID"][index] == "773":
                baseline["ID"][index] = "773684"
            if baseline["ADHDSx_Dx"][index] == "No":
                ADHD_dict[int(baseline["ID"][index])] = 0
            elif baseline["ADHDSx_Dx"][index] == "Yes":
                ADHD_dict[int(baseline["ID"][index])] = 1

In [5]:
def identifyTime(time,weekday):
    t = "23:59:59"
    night = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "07:00:00"
    morning = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "12:00:00"
    afternoon = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "17:00:00"
    evening = datetime.datetime.strptime(t, "%H:%M:%S").time()
    time = time[1:-2]
    time_obj = datetime.datetime.strptime(time, "%H:%M:%S").time()

    if time_obj < morning:
        return "night"
    if time_obj >= morning and time_obj < afternoon:
        return "morning"
    if time_obj >= afternoon and time_obj < evening:
        return "afternoon"
    if time_obj >= evening and time_obj <= night:
        return "evening"

In [6]:
def determineWeekday(date):
    time = date
    time = time[2:-1]
    time = time.replace("T", " ")
    time = time.replace("-", "/")
    datetime_object = datetime.datetime.strptime(time, '%Y/%m/%d %H:%M:%S')
    dayOfTheWeek = datetime_object.weekday()

    if dayOfTheWeek >= 0 and dayOfTheWeek <= 4:
        weekday = 1
    else:
        weekday = 0
    
    return weekday, dayOfTheWeek
    

In [7]:
def CalculateRunning(de_date, index, threshold, first, last, end_of_period, type_activity):
    thr = 0 
    toAdd = 0 

    # check if the element before is "Running"
    if first == 0:
        previousRunning = 0
    else:
        if de_date['Activity'][index-1] != type_activity:
            previousRunning = 0
        else:
            diff = de_date['senseStartTimeMillis'][index] - de_date['senseStartTimeMillis'][index-1]
            if diff > threshold:
                previousRunning = 0
            else:
                previousRunning = 1

    
    # check if the element after is "Running"
    if last == len(de_date['Activity']) - 1:
        afterRunning = 0
    else:
        if de_date['Activity'][index+1] != type_activity:
            afterRunning = 0
        else:
            diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
            if diff > threshold:
                afterRunning = 0
            else:
                afterRunning = 1
    
    # if the following element represent a different activity
    if afterRunning == 0 and last != len(de_date['Activity']) - 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        if diff <= threshold:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
    
    # if the following element represent the same activity
    if afterRunning == 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        toAdd = diff
     
    # if it is the last element  in the dataframe
    if last == len(de_date['Activity']) - 1:
        time = de_date['Final Date'][index] 
        time = time[1:-1]
        time = time.replace("-", "/")
        time += end_of_period
        convert = datetime.datetime.strptime(time,'%Y/%m/%d %H:%M:%S')
        
        epoch_time = convert.timestamp()
        gmt_epoch_time = int(epoch_time - datetime.timezone.utc.utcoffset(convert).total_seconds())*1000 - 4*3600000
        diff = gmt_epoch_time - de_date['senseStartTimeMillis'][index]

        if diff <= threshold:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
        
    return toAdd, thr, previousRunning, afterRunning


In [8]:
Userid = [] 
RunningMorning  = []
RunningAfternoon = []
RunningEvening = []
RunningNight = []
ADHD = []

In [ ]:
#Set the running threshold to be 
#morning: 2 hour 
#afternoon: 2 hour 
#evening: 2 hour
#night: 0 hour 

threshold_night = 0
threshold_morning = 2*3600000
threshold_afternoon = 2*3600000
threshold_evening = 2*3600000


In [9]:

for id in ids:
    # get survey dates for the current id
    surveydates = df.loc[df["Userid"]==id, 'Survey Date'].tolist()
    uniquesurveydates = np.array(surveydates)
    uniquesurveydates = np.unique(uniquesurveydates)
    surveydates = uniquesurveydates.tolist()

    # get a dataframe of the data corresponding to current id
    de_id = df.loc[(df["Userid"])==int(id)]


    # calculate features for each survey date 
    for date in surveydates:
        # retrive dataframe that contains information for current survey date
        de_date = 0
        de_date = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date)]
        # unique dates
        uniq = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date), 'Final Date'].tolist()
        uniq = np.array(uniq)
        uniq = np.unique(uniq)
        uniq = uniq.tolist()
        
        # total time of being Running for a survey date
        totalNightRunning = 0
        totalMorningRunning = 0
        totalAfternoonRunning = 0
        totalEveningRunning = 0
        toAddRunningNight = 0
        toAddRunningMorning = 0
        toAddRunningAfternoon = 0
        toAddRunningEvening = 0
        
        # keep track of first and last
        first = 0
        last = 0
        
        # iterate through dataframe and calculate features        
        for index in de_date.index:
            # determine weekend or weekday
            weekday, dayOfTheWeek = determineWeekday(de_date["senseStartTime"][index])
            
            # determine part of the day 
            partOfDay = identifyTime(de_date["Time"][index],weekday)
            
            if de_date["Activity"][index] == " 'Running'":
                # morning
                if (partOfDay == "morning"):
                    timeRunning, thr, previousRunning, afterRunning = CalculateRunning(de_date, index, threshold_morning, first, last, " 12:00:00", " 'Running'")
                    
                    # If it is a completely new bundle
                    if previousRunning == 0:
                        # add bundle time to total time
                        totalMorningRunning += toAddRunningMorning
                        # reset bundle time
                        toAddRunningMorning  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddRunningMorning += timeRunning
                    # if thr has been reached 
                    else:
                        if toAddRunningMorning < threshold_morning:
                            if timeRunning > threshold_morning:
                                toAddRunningMorning = threshold_morning
                            else:
                                toAddRunningMorning += timeRunning
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddRunningMorning += 600000
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalMorningRunning += toAddRunningMorning
                
                # afternoon
                if (partOfDay == "afternoon"):
                    timeRunning, thr, previousRunning, afterRunning = CalculateRunning(de_date, index, threshold_afternoon, first, last, " 17:00:00", " 'Running'")
                    
                    # If it is a completely new bundle
                    if previousRunning == 0:
                        # add bundle time to total time
                        totalAfternoonRunning += toAddRunningAfternoon
                        # reset bundle time
                        toAddRunningAfternoon  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddRunningAfternoon += timeRunning
                    # if thr has been reached 
                    else:
                        if toAddRunningAfternoon < threshold_afternoon:
                            if timeRunning > threshold_afternoon:
                                toAddRunningAfternoon = threshold_afternoon
                            else:
                                toAddRunningAfternoon += timeRunning
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddRunningAfternoon += 600000
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalAfternoonRunning += toAddRunningAfternoon
                        
                # evening
                if (partOfDay == "evening"):
                    timeRunning, thr, previousRunning, afterRunning = CalculateRunning(de_date, index, threshold_evening, first, last, " 23:59:59", " 'Running'")
                    
                    # If it is a completely new bundle
                    if previousRunning == 0:
                        # add bundle time to total time
                        totalEveningRunning += toAddRunningEvening
                        # reset bundle time
                        toAddRunningEvening  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddRunningEvening += timeRunning
                    # if thr has been reached 
                    else:
                        if toAddRunningEvening < threshold_evening:
                            if timeRunning > threshold_evening:
                                toAddRunningEvening = threshold_evening
                            else:
                                toAddRunningEvening += timeRunning
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddRunningEvening += 600000
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalEveningRunning += toAddRunningEvening
                
                # night
                if (partOfDay == "night"):
                    timeRunning, thr, previousRunning, afterRunning = CalculateRunning(de_date, index, threshold_night, first, last, " 07:00:00", " 'Running'")
                    
                    # If it is a completely new bundle
                    if previousRunning == 0:
                        # add bundle time to total time
                        totalNightRunning += toAddRunningNight
                        # reset bundle time
                        toAddRunningNight  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAddRunningNight += timeRunning
                    # if thr has been reached 
                    else:
                        if toAddRunningNight < threshold_night:
                            if timeRunning > threshold_night:
                                toAddRunningNight = threshold_night
                            else:
                                toAddRunningNight += timeRunning
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAddRunningNight += 600000
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalNightRunning += toAddRunningNight
                    
            # keep track of first and last
            first = 1
            last += 1
        
        # calculate averages
        avgMorningRunning = totalMorningRunning / len(uniq)
        avgAfternoonRunning = totalAfternoonRunning / len(uniq)
        avgEveningRunning = totalEveningRunning / len(uniq)
        avgNightRunning = totalNightRunning / len(uniq)
        
        # append the averages to lists
        Userid.append(id) 
        RunningMorning.append(avgMorningRunning) 
        RunningAfternoon.append(avgAfternoonRunning) 
        RunningEvening.append(avgNightRunning) 
        RunningNight.append(avgNightRunning)
        ADHD.append(ADHD_dict[id])

            
